In [2]:
%matplotlib widget
%config InlineBackend.figure_format = 'svg'

In [3]:
import addict
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from importlib import reload
from tqdm.notebook import tqdm

import celeri
celeri = reload(celeri)

# Plotting the global model is much much faster with tex fonts turned off
plt.rcParams['text.usetex'] = False

In [4]:
command_file_name = "./data/western_north_america/basic_command.json"
command_file_name = "./data/global/global_command.json"
command, segment, block, meshes, station, mogi, sar = celeri.read_data(command_file_name)
station = celeri.process_station(station, command)
segment = celeri.process_segment(segment, command, meshes)
sar = celeri.process_sar(sar, command)
closure, block = celeri.assign_block_labels(segment, station, block, mogi, sar)
# celeri.plot_block_labels(segment, block, station, closure)

In [ ]:
assembly = addict.Dict()
operators = addict.Dict()
assembly = celeri.merge_geodetic_data(assembly, station, sar)
assembly, operators.block_motion_constraints = celeri.block_constraints(assembly, block, command)
assembly, operators.slip_rate_constraints = celeri.slip_rate_constraints(assembly, segment, block, command)
operators.okada_segment_station = celeri.get_segment_station_operator_okada(segment, station, command)
operators.slip_rate_segment_block = celeri.get_fault_slip_rate_partials(segment, block)
assembly.index.sz_elastic = operators.okada_segment_station.shape # Not sure this is correct
assembly.index.sz_slip = operators.slip_rate_segment_block.shape # Not sure this is correct
operators.block_rotation = celeri.get_block_rotation_operator(station, block)
assembly.index.sz_rotation = operators.block_rotation.shape # Not sure this is correct
assembly = celeri.station_row_keep(assembly) # Not sure this is correct


In [ ]:
# celeri.plot_segment_displacements(segment, station, command, segment_idx=0, strike_slip=1, dip_slip=0, tensile_slip=0, lon_min=235, lon_max=255, lat_min=30, lat_max=50, quiver_scale=1e-1)

In [ ]:
model = addict.Dict()

# function [strain, Index, Model] def get_strain_operator(block, station, segment, command, assembly)
"""
Wrapper function for calculating strain partial derivatives.
"""

# [strain, Index.strainBlock]                = deal(zeros(3*numel(Station.lon), 0), []);
# [Model.lonStrain, Model.latStrain]         = deal(zeros(size(Block.interiorLon)));

# TODO: rename block_rotation_flag
# TODO: block.strain_flag for this: Add to matlab converter
# if any(block.strain_flag) # If any blocks have strain turned on
#    if command.strain_method == 1:
#          [strain, Index.strainBlock,...
#           Model.lonStrain, Model.latStrain]   = GetStrainCentroidPartials(Block, Station, Segment);
#    if command.strain_method == 2: # Solve for the reference coordinates
#          [strain, Index.strainBlock]          = GetStrain56Partials(Block, Station, Segment);
#    if command.strain_method == 3: # Solve for the reference latitude only
#          [strain, Index.strainBlock]          = GetStrain34Partials(Block, Station, Segment);
#    if command.strain_method == 4: # Use the simulated annealing approach to solve for reference coordinates inside the block boundaries
#          [strain, Index.strainBlock]          = GetStrainSearchPartials(Block, Station, Segment);
#    end      
# end

# function [G, strainBlockIdx, cLon, cLat, toDelete]  = GetStrainCentroidPartials(Block, Station, Segment)
# % Calculate strain partial derivatives
# nStations                                   = numel(Station.lon);
# nBlocks	                                   = numel(Block.interiorLon);
# G                                           = zeros(3*nStations, 3*nBlocks);
# R              				                 = 6371*1e6; % radius of Earth in mm
# minSta                                      = 3; % 3 for Jack

# % Convert station positions into radians and co-latitude
# Station.lon                                 = deg2rad(Station.lon);
# Station.lat(find(Station.lat)>=0)           = 90 - Station.lat(find(Station.lat)>=0);
# Station.lat(find(Station.lat)<0)            = -90 - Station.lat(find(Station.lat)<0);
# Station.lat                                 = deg2rad(Station.lat);

# % Create array containing column index limits of the design matrix
# firstCol                                    = 3*Station.blockLabel - 2;
# lastCol                                     = 3*Station.blockLabel;

# [cLon, cLat]									 	  = deal(zeros(nBlocks, 1));	

# for iStation = 1:nStations
#    if Block.rotationInfo(Station.blockLabel(iStation)) == 1
#       % Partials with block centroid
#       % The block "centroid" is only an approximation given by the mean of its coordinates
#       % This should work reasonably well for the "chopped" case but is not exact or general
#       idx1                                  = find(Segment.eastLabel == Station.blockLabel(iStation));
#       idx2                                  = find(Segment.westLabel == Station.blockLabel(iStation));
#       idx                                   = union(idx1, idx2);
#       lonVec                                = [Segment.lon1(idx);Segment.lon2(idx)];
#       latVec                                = [Segment.lat1(idx);Segment.lat2(idx)];
# %       [lat0 lon0]                           = meanm(latVec(:), lonVec(:));
#       lat0                                  = mean(latVec(:));
#       lon0                                  = mean(lonVec(:));
#       cLon(Station.blockLabel(iStation))	  = lon0;
#       cLat(Station.blockLabel(iStation))	  = lat0;
#       lon0                                  = deg2rad(lon0);
#       lat0(lat0>=0)                         = 90 - lat0(lat0>=0);
#       lat0(lat0<0)                          = -90 - lat0(lat0<0);      
#       lat0                                  = deg2rad(lat0);
#       G(3*iStation-2:3*iStation, firstCol(iStation):lastCol(iStation)) = [R*(Station.lon(iStation)-lon0).*sin(lat0) R*(Station.lat(iStation)-lat0) 0;...
#                                                                           0 R*(Station.lon(iStation)-lon0).*sin(lat0) R*(Station.lat(iStation)-lat0);...
#                                                                           0 0 0];
#    end
# end

# % Get rid of blocks that don't have enough stations on them
# for i = 1:size(G, 2)/3;
#    j                                        = 3*i-1;
#    numSta(i)                                = numel(find(G(:, j)));
# end
# toDelete                                    = [3*find(numSta<2*minSta)-2 3*find(numSta<2*minSta)-1 3*find(numSta<2*minSta)];
# strainBlockIdx                              = setdiff(1:1:nBlocks*3, toDelete);
# G(:,toDelete)                               = [];





In [18]:
with open('test.npy', 'wb') as f:
    np.save(f, closure.polygons)
with open('test.npy', 'rb') as f:
    saved_closure = np.load(f)

len(closure.polygons)

ValueError: Object arrays cannot be loaded when allow_pickle=False

In [32]:
closure.polygons[0].edge_idxs

[0,
 2,
 1320,
 1323,
 1324,
 1330,
 1332,
 1334,
 1364,
 1382,
 1326,
 1328,
 1502,
 1500,
 1498,
 1496,
 1494,
 1490,
 1492,
 1506,
 1504,
 17,
 19,
 21,
 22,
 24,
 1588,
 1586,
 1584,
 240,
 624,
 608,
 626,
 622,
 620,
 618,
 114,
 112,
 706,
 218,
 584,
 582,
 580,
 578,
 1040,
 1044,
 978,
 1354,
 1385,
 1315,
 1317,
 1319,
 1285,
 1282,
 1281,
 1279,
 1277,
 1274,
 34,
 32,
 30]